In [ ]:
#Homogenizar catálogos y unirlos
import pandas as pd
import geopandas

In [ ]:
#1. Set the path of both catalogs /// Path de catálogos
catalogo_1993_2018 = "C:/Users/sophi/Documents/Trabajos/SGC/test_sgc/cat_1993.xlsx"
catalogo_2018_hoy = "C:/Users/sophi/Documents/Trabajos/SGC/test_sgc/cat_2018.xlsx"

#Read catalogs
catinicial_1993 = pd.read_excel(catalogo_1993_2018)
catinicial_2018 = pd.read_excel(catalogo_2018_hoy, skiprows=13)

In [ ]:
#Definir estructura del catálogo final // Define final catalog structure
out_columns = ["fecha", "dia", "mes", "ano", "lat", "long", "prof", "mag_mi", "mag_mw", "mag_u",
               "tipo_mag", "fases", "rms", "gap", "pais", "dep", "mun", "er_lat", "er_long", "er_prof", "estado", "cat"]

#Crear DataFrame para mejorar estructura
cat_final_1993 = pd.DataFrame(columns = out_columns)
cat_final_2018 = pd.DataFrame(columns = out_columns)

In [ ]:
#Renombra columnas de cada uno respecto a catálogo final // Rename columns to final structure names
dict_rename_columns_cat_2018 = {'FECHA - HORA UTC': "fecha", 'LATITUD (°)': "lat", 'LONGITUD (°)': "long",
       'PROF. (Km)': "prof", 'MAGNITUD': "mag_u", "TIPO MAGNITUD": "tipo_mag", 'FASES': "fases", 'RMS (Seg)': "rms", 'GAP (°)': "gap",
       'ERROR LATITUD (Km)': "er_lat", 'ERROR LONGITUD (Km)': "er_long", 'ERROR PROFUNDIDAD (Km)':"er_prof",
       'REGION': 'region', 'ESTADO': "estado"}

catinicial_2018 = catinicial_2018.rename(columns=dict_rename_columns_cat_2018)

#Rename columns as they are called in the out shp.
dict_rename_columns_cat_1993 = {'FECHA': "fecha", 'HORA_UTC': "hora_utc", 'LATITUD (grados)': "lat", 'LONGITUD (grados)': "long",
       'PROFUNDIDAD (Km)': "prof", 'MAGNITUD Ml': "mag_mi", 'MAGNITUD Mw': "mag_mw", 'DEPARTAMENTO': "dep",
       'MUNICIPIO': "mun", '# FASES': "fases", 'RMS (Seg)': "rms", 'GAP (grados)': "gap",
       'ERROR LATITUD (Km)': "er_lat", 'ERROR LONGITUD (Km)': "er_long", 'ERROR PROFUNDIDAD (Km)':"er_prof",
       'ESTADO': "estado"}

catinicial_1993 = catinicial_1993.rename(columns=dict_rename_columns_cat_1993)

In [ ]:
cat_final_1993 = cat_final_1993.append(catinicial_1993)
cat_final_2018 = cat_final_2018.append(catinicial_2018)

In [ ]:
def insertar_diamesano(cat):
    cat.fecha = pd.to_datetime(cat['fecha'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    cat.dia = cat["fecha"].dt.day
    cat.mes = cat["fecha"].dt.month
    cat.ano = cat["fecha"].dt.year
    
insertar_diamesano(cat_final_1993); insertar_diamesano(cat_final_2018)
cat_final_1993["cat"] = "1"; cat_final_2018["cat"] = "2"

In [ ]:
#Asignar al catálogo de 1993 la magnitud a usar "mag_u" y su tipo
cat_final_1993.mag_u = cat_final_1993.mag_mi
cat_final_1993.tipo_mag = "Mi"

cat_final_1993["mag_u"] = cat_final_1993["mag_mw"].where(cat_final_1993['mag_mw'].notna(), cat_final_1993["mag_u"])
cat_final_1993["tipo_mag"] = cat_final_1993["tipo_mag"].mask(cat_final_1993['mag_mw'].notna(), "Mw")
cat_final_1993.loc[cat_final_1993['mag_mw'].notna() == True]
list(cat_final_1993.tipo_mag)

cat_final_1993["region"] = cat_final_1993["mun"].str.capitalize() + ", " + cat_final_1993["dep"].str.capitalize()

#Dividir en columnas municipio, departamento y país en el catálogo 2018
cat_final_2018[["mun", "pais"]] = cat_final_2018["region"].str.split(", ", expand = True)
cat_final_2018[["mun", "dep"]] = cat_final_2018["mun"].str.split(" - ", expand = True)
cat_final_2018.pais = cat_final_2018.pais.str.strip(); cat_final_2018.mun = cat_final_2018.mun.str.strip(); cat_final_2018.dep = cat_final_2018.dep.str.strip()


In [ ]:
cat_unido = pd.concat([cat_final_1993, cat_final_2018], axis = 0)
cat_unido.columns

In [ ]:
#Seleccionar columnas para exportar a shape
cat_to_shp = cat_unido[['fecha', 'lat', 'long', 'prof', 'region', 'mag_u', 'tipo_mag',
        'estado', 'cat']]
cat_to_shp

In [ ]:
geocat = geopandas.GeoDataFrame(
    cat_to_shp, geometry=geopandas.points_from_xy(cat_to_shp.long, cat_to_shp.lat), crs="EPSG:4326")

geocat_proj = geocat.to_crs(9377)

In [ ]:
#asignar nombre
fecha_min = cat_to_shp['fecha'].dt.date.min().strftime('%d%m%Y')
fecha_max = cat_to_shp['fecha'].dt.date.max().strftime('%d%m%Y')
file_name = "cat_proj_" + fecha_min + "_" + fecha_max 
geocat_proj.to_file('D:/' + file_name, driver='ESRI Shapefile', index = True)
cat_unido.to_excel('D:/' + file_name + ".xlsx")